In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .appName("My Spark Application") \
    .master("local[2]") \
    .config("spark.executor.memory", "2g") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

In [3]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType

schema = StructType([
    StructField("category", StringType(), False),    
    StructField("description", StringType(), True),       
    StructField("id", LongType(), False),                 
    StructField("image", StringType(), True),             
    StructField("price", DoubleType(), False),              
    StructField("rating", StructType([                     
        StructField("count", LongType(), True),            
        StructField("rate", DoubleType(), True)            
    ]), True),
    StructField("title", StringType(), False)               
])

In [4]:
df = spark.read.json("fake_store_products.json", schema=schema)

In [5]:
df.show()

+----------------+--------------------+---+--------------------+------+----------+--------------------+
|        category|         description| id|               image| price|    rating|               title|
+----------------+--------------------+---+--------------------+------+----------+--------------------+
|  men's clothing|Your perfect pack...|  1|https://fakestore...|109.95|{120, 3.9}|Fjallraven - Fold...|
|  men's clothing|Slim-fitting styl...|  2|https://fakestore...|  22.3|{259, 4.1}|Mens Casual Premi...|
|  men's clothing|great outerwear j...|  3|https://fakestore...| 55.99|{500, 4.7}|  Mens Cotton Jacket|
|  men's clothing|The color could b...|  4|https://fakestore...| 15.99|{430, 2.1}|Mens Casual Slim Fit|
|        jewelery|From our Legends ...|  5|https://fakestore...| 695.0|{400, 4.6}|John Hardy Women'...|
|        jewelery|Satisfaction Guar...|  6|https://fakestore...| 168.0| {70, 3.9}|Solid Gold Petite...|
|        jewelery|Classic Created W...|  7|https://fakestore...|

In [6]:
df.printSchema()

root
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- id: long (nullable = true)
 |-- image: string (nullable = true)
 |-- price: double (nullable = true)
 |-- rating: struct (nullable = true)
 |    |-- count: long (nullable = true)
 |    |-- rate: double (nullable = true)
 |-- title: string (nullable = true)



In [7]:
from pyspark.sql import functions as F
def transform(df):
    usd_to_try=34.66
    df = df.withColumn("review", F.col("rating.count")).withColumn("score", F.col("rating.rate"))
    df = df.withColumn("price", F.col("price").cast("double"))
    df = df.withColumn("price_try", F.round(F.col("price") * usd_to_try,0))
    df = df.drop("rating")
    df = df.drop("price")
    return df
transformed_data = transform(df)

In [8]:
transformed_data.show(5)
transformed_data.printSchema()

+--------------+--------------------+---+--------------------+--------------------+------+-----+---------+
|      category|         description| id|               image|               title|review|score|price_try|
+--------------+--------------------+---+--------------------+--------------------+------+-----+---------+
|men's clothing|Your perfect pack...|  1|https://fakestore...|Fjallraven - Fold...|   120|  3.9|   3811.0|
|men's clothing|Slim-fitting styl...|  2|https://fakestore...|Mens Casual Premi...|   259|  4.1|    773.0|
|men's clothing|great outerwear j...|  3|https://fakestore...|  Mens Cotton Jacket|   500|  4.7|   1941.0|
|men's clothing|The color could b...|  4|https://fakestore...|Mens Casual Slim Fit|   430|  2.1|    554.0|
|      jewelery|From our Legends ...|  5|https://fakestore...|John Hardy Women'...|   400|  4.6|  24089.0|
+--------------+--------------------+---+--------------------+--------------------+------+-----+---------+
only showing top 5 rows

root
 |-- ca

In [9]:
## SparkSQL - Queries

In [10]:
transformed_data.createTempView("store")

In [11]:
# Total product numbers in store
spark.sql("""
SELECT COUNT(*) as TOTAL_PRODUCT_NUMBER FROM store""").show()

+--------------------+
|TOTAL_PRODUCT_NUMBER|
+--------------------+
|                  20|
+--------------------+



In [12]:
# Total product numbers Grouped by Categories
spark.sql("""
SELECT category, COUNT(*) TOTAL_PRODUCT FROM store
GROUP BY category""").show()

+----------------+-------------+
|        category|TOTAL_PRODUCT|
+----------------+-------------+
|  men's clothing|            4|
|women's clothing|            6|
|        jewelery|            4|
|     electronics|            6|
+----------------+-------------+



In [13]:
# Calculate Average Review, Average Price, and Average Score Grouped by Category
spark.sql("""
SELECT 
    category,
    round(AVG(review)) as AVG_REVIEW,
    round(AVG(score)) as AVG_SCORE,
    round(AVG(price_try)) as AVG_PRICE_TRY
FROM store
GROUP BY category""").show()

+----------------+----------+---------+-------------+
|        category|AVG_REVIEW|AVG_SCORE|AVG_PRICE_TRY|
+----------------+----------+---------+-------------+
|  men's clothing|     327.0|      4.0|       1770.0|
|women's clothing|     279.0|      4.0|        911.0|
|        jewelery|     243.0|      3.0|       7660.0|
|     electronics|     297.0|      3.0|      11524.0|
+----------------+----------+---------+-------------+



In [14]:
# The most expensive product along with all its details.
spark.sql("""
SELECT
    category, description, title, review, score, price_try
FROM store
WHERE price_try IN(SELECT MAX(price_try) FROM store)""").show()

+-----------+--------------------+--------------------+------+-----+---------+
|   category|         description|               title|review|score|price_try|
+-----------+--------------------+--------------------+------+-----+---------+
|electronics|49 INCH SUPER ULT...|Samsung 49-Inch C...|   140|  2.2|  34660.0|
+-----------+--------------------+--------------------+------+-----+---------+



In [15]:
# The most affordable product along with all its details.
spark.sql("""
SELECT
    category, description, title, review, score, price_try
FROM store
WHERE price_try IN(SELECT MIN(price_try) FROM store)""").show()

+----------------+--------------------+--------------------+------+-----+---------+
|        category|         description|               title|review|score|price_try|
+----------------+--------------------+--------------------+------+-----+---------+
|women's clothing|100% Polyester, M...|Opna Women's Shor...|   146|  4.5|    276.0|
+----------------+--------------------+--------------------+------+-----+---------+



In [16]:
# The most expensive product for each category along with all its details.
spark.sql("""
SELECT
    s.category,
    s.description,
    s.title,
    s.review,
    s.score,
    s.price_try
FROM store s
INNER JOIN (
    SELECT 
        category, 
        MAX(price_try) AS max_price 
    FROM store 
    GROUP BY category
) max_prices
ON s.category = max_prices.category AND s.price_try = max_prices.max_price
""").show()

+----------------+--------------------+--------------------+------+-----+---------+
|        category|         description|               title|review|score|price_try|
+----------------+--------------------+--------------------+------+-----+---------+
|  men's clothing|Your perfect pack...|Fjallraven - Fold...|   120|  3.9|   3811.0|
|        jewelery|From our Legends ...|John Hardy Women'...|   400|  4.6|  24089.0|
|     electronics|49 INCH SUPER ULT...|Samsung 49-Inch C...|   140|  2.2|  34660.0|
|women's clothing|Note:The Jackets ...|BIYLACLESEN Women...|   235|  2.6|   1975.0|
+----------------+--------------------+--------------------+------+-----+---------+

